# A Python Guide to Buying Pullbacks, Not Breakouts - Analytics

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2024-10-25 09:20:00,1338.4,1338.6,1337.4,1337.7,4128
2024-10-25 09:25:00,1337.7,1338.0,1337.0,1337.7,2796
2024-10-25 09:30:00,1337.7,1338.0,1335.5,1336.2,4931


In [5]:
short_term_ma_periods = [10, 11, 12, 13, 14, 15]
best_return = 0
best_short_ma = None

for short_ma_period in short_term_ma_periods:
    # Calculate the short-term and long-term moving averages
    data['short_ma'] = data['Close'].rolling(window=short_ma_period).mean()
    data['long_ma'] = data['Close'].rolling(window=200).mean()

    # Generate trading signals
    data['signal'] = 0
    data.loc[(data['Close'] < data['short_ma']) & (data['Close'] > data['long_ma']), 'signal'] = 1
    data.loc[data['Close'] > data['short_ma'], 'signal'] = -1

    # Backtest the strategy
    data['position'] = data['signal'].shift()
    data = data.dropna()

    # Calculate returns
    data['return'] = data['position'] * data['Close'].pct_change()

    # Calculate strategy metrics
    average_return_per_trade = data['return'].mean()
    holding_period_days = data['position'].abs().sum()
    sharpe_ratio = (data['return'].mean() / data['return'].std()) * (252 ** 0.5)  # Assuming 252 trading days in a year

    # Update best short-term moving average if the return is higher
    if average_return_per_trade > best_return:
        best_return = average_return_per_trade
        best_short_ma = short_ma_period

In [6]:
# Print the optimal short-term moving average and its metrics
print("Optimal Short-Term Moving Average:", best_short_ma)
print("Average Return per Trade:", best_return)
print("Holding Period (in days):", holding_period_days)
print("Sharpe Ratio:", sharpe_ratio)

Optimal Short-Term Moving Average: None
Average Return per Trade: 0
Holding Period (in days): 56981.0
Sharpe Ratio: -0.3107909410623742
